In [2]:
%cd /content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1/

/content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1


In [2]:
import numpy as np
import data_utils
import implementations
%load_ext autoreload
%autoreload 2

In [3]:
train_data, train_label = data_utils.load_data('Data/train.csv')
train_data.shape, train_label.shape

((250000, 30), (250000,))

In [4]:
def cross_validation(train_data, train_label, k_fold, eval_func, input_dict):
  data_num = train_data.shape[0]
  data_dim = train_data.shape[1]
  fold_part_idx = data_utils.k_fold(data_num, k_fold)

  loss_list = []
  acc_list = []
  for k in range(k_fold): 
    cur_train_idx = np.concatenate([fold_part_idx[:k, :], fold_part_idx[k+1:, :]], axis = 0)
    cur_train_idx = cur_train_idx.flatten()
    cur_test_idx = fold_part_idx[k, :]
    x_tr = train_data[cur_train_idx, :]
    x_te = train_data[cur_test_idx, :]
    y_tr = train_label[cur_train_idx]
    y_te = train_label[cur_test_idx]

    w, loss = eval_func(y_tr, x_tr,  **input_dict)
    _, train_acc = implementations.compute_statistics_all(y_tr, x_tr, w, func_type = input_dict['func_type'])
    test_loss, test_acc = implementations.compute_statistics_all(y_te, x_te, w, func_type = input_dict['func_type'])
    loss_list.append(test_loss)
    acc_list.append(test_acc)
    print(test_acc)
  return np.mean(loss_list), np.mean(acc_list)

In [5]:
group_num = 2
k_fold_num = 5

In [23]:
%autoreload 2

group_num = 0
max_acc = 0
lambdas = np.logspace(-10, 0, 10)
gammas = [0.016]
iters = [10000]
test_func = implementations.mean_squared_error_sgd
for clean in [1]:
  for poly in [1]:
    if(poly):
      for degree in [3, 4, 5, 6, 7, 8, 9]:
        degree_list = [1] * 3
        degree_list[group_num] = degree
        for normalize in [1]:
          for all_poly in [True]:
            group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                          data_utils.process_data(train_data, train_label, clean = clean)
            group0_x, group1_x, group2_x = \
                    data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
            cur_group = [group0_x, group1_x, group2_x]
            cur_labels = [group0_labels, group1_labels, group2_labels]
            group_x = cur_group[group_num]
            group_label = cur_labels[group_num]
            if(normalize):
              group_x = data_utils.normalize(group_x)
            
            for gamma in gammas:
              for iteration in iters:
                input_dict = {'lambda_':0.00000001, \
                        'initial_w':np.zeros(group_x.shape[1]), \
                        'max_iters':iteration, \
                        'gamma':gamma, \
                        'sgd':False, \
                        'func_type':'linear'}
                
                loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
                if(acc > max_acc):
                  print(f'Found a better scheme. \
                      Clean:{clean}, \
                      Poly:{poly}, \
                      Degree:{degree}, \
                      Normalize: {normalize}, \
                      all_poly:{all_poly}, \
                      gamma:{gamma},\
                      iteration:{iteration},\
                      Acc:{acc}')
                  max_acc = acc
    else:
      for normalize in [0, 1]:
        group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                      data_utils.process_data(train_data, train_label, clean = clean)
        cur_group = [group0_x, group1_x, group2_x]
        cur_labels = [group0_labels, group1_labels, group2_labels]
        group_x = cur_group[group_num]
        group_label = cur_labels[group_num]
        if(normalize):
          group_x = data_utils.normalize(group_x)
        for gamma in gammas:
          for iteration in iters:
            input_dict = {'lambda_':0.01, \
                    'initial_w':np.zeros(group_x.shape[1]), \
                    'max_iters':iteration, \
                    'gamma':gamma, \
                    'sgd':False, \
                    'func_type':'linear'}
            loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
            if(acc > max_acc):
              print(f'Found a better scheme. \
                  Clean:{clean}, \
                  Poly:{poly}, \
                  Degree:No, \
                  Normalize: {normalize}, \
                  gamma:{gamma},\
                  iteration:{iteration},\
                  Acc:{acc}')
              max_acc = acc

0.8305474927434692
0.8258432589330398
0.8112801521369233
0.8223401060954859
0.8291462316084476
Found a better scheme.                       Clean:1,                       Poly:1,                       Degree:3,                       Normalize: 1,                       all_poly:True,                       gamma:0.016,                      iteration:10000,                      Acc:0.8238314483034731
0.8337003302972675
0.8296466820138124
0.8079271344209789
0.8239915924331899
0.8298969072164949
Found a better scheme.                       Clean:1,                       Poly:1,                       Degree:4,                       Normalize: 1,                       all_poly:True,                       gamma:0.016,                      iteration:10000,                      Acc:0.8250325292763486
0.8337003302972675
0.8285456911220098
0.80307276548894
0.8239915924331899
0.8302472225002502
0.8325993394054649
0.8271444299869883
0.7964167750975878
0.8247422680412371


KeyboardInterrupt: 